<a href="https://colab.research.google.com/github/mcppp/FinalProjectI2DL/blob/main/Extract_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import sys
import librosa
from google.colab import drive
import soundfile
import matplotlib.pyplot as plt


from os import listdir
from os.path import isfile, join
import os
import subprocess
import sys

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
project_path = './drive/MyDrive/Colab Notebooks/Project'
dataset_path = join(project_path, 'Dataset')

In [ ]:
feature_all=np.empty((0,193)) #for d=193 features that will be extracted in total 
labels = list()

dir = os.listdir(dataset_path)

for i in dir:
  audiofile = os.listdir(join(dataset_path,i)) #audiofile has a list of the 60 audios produced by one actor (in folder i, included in dir)
  for j in audiofile: #for every audiofile
    labels.append(j[7]) #label is in position 7 of the audio filename 

    time_series,sr = librosa.load(join(dataset_path,i,j)) #audio time series, sampling rate of it, shape: (83139,)
    stft = np.abs(librosa.stft(time_series)) #FFT and absolute value, shape: (1025, 163)

    mfccs = np.mean(librosa.feature.mfcc(y=time_series, sr=sr, n_mfcc=40).T,axis=0) #returns (mean) of 40 MFC coefficients
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sr).T,axis=0) #chromagram from a waveform or power spectrogram
    mel = np.mean(librosa.feature.melspectrogram(y=time_series, sr=sr).T,axis=0) #mel-scaled spectrogram
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sr).T,axis=0) #spectral contrast
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(time_series),sr=sr).T,axis=0) #tonal centroid features

    features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
    #print(features.shape) #(193,) the features of one sample in a row (d=193)

    feature_all = np.vstack([feature_all,features]) #stacking samples in a matrix that will be of size nxd (will grow if we loop through all audio files)
    #print(feature_all.shape) 
  
labels = np.array(labels)

/usr/local/lib/python3.6/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=1012
  n_fft, y.shape[-1]


In [ ]:
##Saving extracted features and labels into .npy arrays for later use
np.save(join(project_path,'feature_matrix'),feature_all)
np.save(join(project_path,'labels'),labels)    

In [ ]:
##Checking if features were properly saved and can be loaded again
feature_matrix=np.load(join(project_path, 'feature_matrix.npy'))
sample_labels=np.load(join(project_path, 'labels.npy'))

print(feature_matrix.shape) #(1440, 193)
print(sample_labels.shape) #(1440,)

(1440, 193)
(1440,)
